### 필요 패캐지

In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import FinanceDataReader as fdr
from scipy.optimize import minimize
from itertools import combinations

### 1. 데이터 수집

In [2]:
df_kospi = fdr.StockListing('KRX')
df_etf = fdr.StockListing('ETF/KR')
df_customer = pd.read_csv('data/customer_dummy.csv')

In [3]:
## 고객데이터 칼럼이름 지정 및 데이터 타입 변환
df_customer.columns = ['customer_id', 'stock_name', 'invested_value']
df_customer['invested_value'] = df_customer['invested_value'].astype(float) 
df_customer['stock_name'] = df_customer['stock_name'].replace('네이버', 'NAVER')

### 2. ETF 구성 종목 정보 수집

In [4]:
def get_etf_components(code):
 
    try:
        url = f"https://navercomp.wisereport.co.kr/v2/ETF/index.aspx?cmp_cd={code}"
        html = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(html.text, "lxml")
        soup = list(soup)[1]

        start_str = str(soup).index("grid_data")
        end_str = str(soup).index("chartDraw")
        soup_list = str(soup)[start_str+12:end_str]

        CU_list = []
        if len(soup_list) > 1:
            if "chart_data" in soup_list:
                end_str = soup_list.index("chart_data")
                soup_list = soup_list[:end_str - 3]

            while True:
                idx = soup_list.find('}')
                if idx == -1:
                    break
                name = soup_list[1:idx]
                soup_list = soup_list[idx+2:]
                CU_list.append(name)
                if len(soup_list) < 10:
                    break

            CU_df = pd.DataFrame(CU_list)
            cu_tmp = CU_df[0].str.split(",")

            CU_main = pd.DataFrame()
            for i in range(4):
                CU_main = pd.concat([CU_main, cu_tmp.str[i].str.split(":").str[1]], axis=1)

            CU_main.columns = ["TRD_DT", "AGMT_STK_CNT", "STK_NM_KOR", "ETF_WEIGHT"]
            CU_main.insert(0, "CODE", str(code))

            # 전처리
            CU_main["TRD_DT"] = CU_main["TRD_DT"].str.replace('"', '')
            CU_main["STK_NM_KOR"] = CU_main["STK_NM_KOR"].str.replace('"', '')
            CU_main["AGMT_STK_CNT"] = CU_main["AGMT_STK_CNT"].replace("null", "0").str.replace('.', '', regex=False)
            CU_main["ETF_WEIGHT"] = CU_main["ETF_WEIGHT"].replace("null", "0").str.replace('.', '', regex=False)

            CU_main["AGMT_STK_CNT"] = CU_main["AGMT_STK_CNT"].astype(int) / 100
            CU_main["ETF_WEIGHT"] = CU_main["ETF_WEIGHT"].astype(int) / 100

            return CU_main

        else:
            print(f"⚠️ 구성 종목 없음: {code}")
            return None

    except Exception as e:
        print(f"❌ 에러 발생 ({code}): {e}")
        return None

In [5]:
result_list = []

for code in df_etf["Symbol"]:
    print(f"🔍 {code} 처리 중...")
    df = get_etf_components(code)
    if df is not None:
        result_list.append(df)

df_etf_cu = pd.concat(result_list, ignore_index=True)
print(df_etf_cu.head())

🔍 360750 처리 중...
🔍 459580 처리 중...
🔍 488770 처리 중...
🔍 069500 처리 중...
🔍 133690 처리 중...
🔍 379800 처리 중...
🔍 357870 처리 중...
🔍 423160 처리 중...
🔍 273130 처리 중...
🔍 381170 처리 중...
🔍 102110 처리 중...
🔍 278530 처리 중...
🔍 379810 처리 중...
🔍 449170 처리 중...
🔍 122630 처리 중...
🔍 381180 처리 중...
🔍 455890 처리 중...
🔍 453850 처리 중...
🔍 481050 처리 중...
🔍 360200 처리 중...
🔍 458730 처리 중...
🔍 0043B0 처리 중...
🔍 233740 처리 중...
🔍 453540 처리 중...
🔍 367380 처리 중...
🔍 385540 처리 중...
🔍 310970 처리 중...
🔍 371460 처리 중...
🔍 214980 처리 중...
🔍 161510 처리 중...
🔍 148020 처리 중...
🔍 102780 처리 중...
🔍 252670 처리 중...
🔍 476550 처리 중...
🔍 494900 처리 중...
🔍 449450 처리 중...
🔍 229200 처리 중...
🔍 411060 처리 중...
🔍 466920 처리 중...
🔍 292150 처리 중...
🔍 305720 처리 중...
🔍 379780 처리 중...
🔍 368590 처리 중...
🔍 477080 처리 중...
🔍 479080 처리 중...
🔍 0061Z0 처리 중...
🔍 371160 처리 중...
🔍 451540 처리 중...
🔍 294400 처리 중...
🔍 491080 처리 중...
🔍 458760 처리 중...
🔍 475630 처리 중...
🔍 436140 처리 중...
🔍 305540 처리 중...
🔍 469830 처리 중...
🔍 457480 처리 중...
🔍 157450 처리 중...
🔍 315930 처리 중...
🔍 356540 처리 중.

### 3. 테마 및 산업 정보 수집


In [6]:
headers = {'User-Agent': 'Mozilla/5.0'}

# 1. 전체 테마 이름 및 번호 크롤링
def get_theme_list():
    url = "https://finance.naver.com/sise/theme.naver"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "lxml")

    theme_list = []

    for a in soup.select("td.col_type1 > a"):
        theme_name = a.text.strip()
        href = a['href']
        if "no=" in href:
            no = href.split("no=")[-1]
            theme_list.append({"theme_name": theme_name, "theme_no": int(no)})

    return pd.DataFrame(theme_list)

# 2. 특정 테마 번호로 구성 종목 크롤링
def get_theme_stocks(theme_name, theme_no):
    url = f"https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no={theme_no}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "lxml")

    table = soup.select_one("table.type_5")
    if table is None:
        return pd.DataFrame()

    rows = table.select("tr")[2:]  # 첫 2줄은 헤더 or 공백

    stocks = []
    for row in rows:
        cols = row.select("td")
        if len(cols) < 2:
            continue
        name_tag = cols[0].select_one("a")
        if not name_tag:
            continue
        stock_name = name_tag.text.strip()
        href = name_tag['href']
        if "code=" in href:
            stock_code = href.split("code=")[-1]
            current_price = cols[1].text.strip().replace(",", "")
            stocks.append({
                "theme_name": theme_name,
                "theme_no": theme_no,
                "stock_name": stock_name,
                "stock_code": stock_code,
                "current_price": current_price
            })

    return pd.DataFrame(stocks)

# 3. 전체 테마 반복 수집
def collect_all_theme_stocks():
    theme_df = get_theme_list()
    all_stocks = []

    for _, row in theme_df.iterrows():
        print(f"크롤링 중: {row['theme_name']} (no={row['theme_no']})")
        try:
            df = get_theme_stocks(row["theme_name"], row["theme_no"])
            all_stocks.append(df)
            time.sleep(0.3)  # 서버 차단 방지용
        except Exception as e:
            print(f"❌ 에러 발생: {e}")

    return pd.concat(all_stocks, ignore_index=True)

# 실행
df_kospi_theme_stocks = collect_all_theme_stocks()

크롤링 중: 2차전지(생산) (no=449)
크롤링 중: MLCC(적층세라믹콘덴서) (no=405)
크롤링 중: 스마트카(SMART CAR) (no=332)
크롤링 중: 자동차 대표주 (no=159)
크롤링 중: 온디바이스 AI (no=545)
크롤링 중: 정유 (no=185)
크롤링 중: 리튬 (no=523)
크롤링 중: 2차전지(전고체) (no=472)
크롤링 중: 2차전지(나트륨이온) (no=579)
크롤링 중: IT 대표주 (no=173)
크롤링 중: 전기차 (no=227)
크롤링 중: 카메라모듈/부품 (no=41)
크롤링 중: 바이오인식(생체인식) (no=106)
크롤링 중: 2차전지(LFP/리튬인산철) (no=503)
크롤링 중: 마이코플라스마 폐렴 (no=546)
크롤링 중: 맥신(MXene) (no=539)
크롤링 중: 시멘트/레미콘 (no=44)
크롤링 중: 4대강 복원 (no=374)
크롤링 중: 그래핀 (no=415)
크롤링 중: 윤활유 (no=527)
크롤링 중: 아이폰 (no=388)
크롤링 중: 무선충전기술 (no=321)
크롤링 중: 전기자전거 (no=268)
크롤링 중: 콜드체인(저온 유통) (no=464)
크롤링 중: PCB(FPCB 등) (no=287)
크롤링 중: 스마트폰 (no=279)
크롤링 중: 자율주행차 (no=362)
크롤링 중: 2차전지(소재/부품) (no=446)
크롤링 중: 리비안(RIVIAN) (no=501)
크롤링 중: 2차전지 (no=64)
크롤링 중: CXL(컴퓨트익스프레스링크) (no=547)
크롤링 중: 태풍 및 장마 (no=82)
크롤링 중: 시스템반도체 (no=307)
크롤링 중: 메르스 코로나 바이러스 (no=346)
크롤링 중: 반도체 재료/부품 (no=14)
크롤링 중: 2차전지(장비) (no=445)
크롤링 중: 재난/안전(지진/화재 등) (no=335)
크롤링 중: 3D 낸드(NAND) (no=370)
크롤링 중: 통신 (no=126)
크롤링 중: 갤럭시 부품주 (no=393)


In [7]:
headers = {'User-Agent': 'Mozilla/5.0'}

# 1. 전체 업종 이름 및 번호 크롤링
def get_industry_list():
    url = "https://finance.naver.com/sise/sise_group.naver?type=upjong"
    res = requests.get(url, headers=headers)
    res.encoding = 'euc-kr'
    soup = BeautifulSoup(res.text, "lxml")

    industry_list = []

    for a in soup.select("td > a"):
        industry_name = a.text.strip()
        href = a['href']
        if "no=" in href:
            no = href.split("no=")[-1]
            industry_list.append({"industry_name": industry_name, "industry_no": int(no)})

    return pd.DataFrame(industry_list)

# 2. 특정 업종 번호로 구성 종목 크롤링
def get_industry_stocks(industry_name, industry_no):
    url = f"https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no={industry_no}"
    res = requests.get(url, headers=headers)
    res.encoding = 'euc-kr'
    soup = BeautifulSoup(res.text, "lxml")

    table = soup.find("table", class_="type_5")
    if not table:
        return pd.DataFrame()

    rows = table.find_all("tr")[2:]  # 헤더 제외
    stocks = []

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 2:
            continue

        name_tag = cols[0].find("a")
        if not name_tag:
            continue

        stock_name = name_tag.text.strip()
        href = name_tag.get("href", "")
        if "code=" not in href:
            continue
        stock_code = href.split("code=")[-1]
        current_price = cols[1].text.strip().replace(",", "")

        stocks.append({
            "industry_name": industry_name,
            "industry_no": industry_no,
            "stock_name": stock_name,
            "stock_code": stock_code,
            "current_price": current_price
        })

    return pd.DataFrame(stocks)

# 3. 전체 업종 반복 수집
def collect_all_industry_stocks():
    industry_df = get_industry_list()
    all_stocks = []

    for _, row in industry_df.iterrows():
        print(f"🔍 크롤링 중: {row['industry_name']} (no={row['industry_no']})")
        try:
            df = get_industry_stocks(row["industry_name"], row["industry_no"])
            if not df.empty:
                all_stocks.append(df)
                print(f"✅ {row['industry_name']} - {len(df)}개 종목 수집됨")
            else:
                print(f"⚠️ {row['industry_name']} 업종은 종목 없음")
            time.sleep(0.3)
        except Exception as e:
            print(f"❌ 에러 발생: {e}")

    if all_stocks:
        return pd.concat(all_stocks, ignore_index=True)
    else:
        print("❗ 유효한 업종 종목이 없습니다.")
        return pd.DataFrame()

# 실행
df_kospi_industry_stocks = collect_all_industry_stocks()

🔍 크롤링 중: 전자장비와기기 (no=282)
✅ 전자장비와기기 - 111개 종목 수집됨
🔍 크롤링 중: 자동차 (no=273)
✅ 자동차 - 14개 종목 수집됨
🔍 크롤링 중: 자동차부품 (no=270)
✅ 자동차부품 - 153개 종목 수집됨
🔍 크롤링 중: 반도체와반도체장비 (no=278)
✅ 반도체와반도체장비 - 165개 종목 수집됨
🔍 크롤링 중: 무선통신서비스 (no=333)
✅ 무선통신서비스 - 4개 종목 수집됨
🔍 크롤링 중: 전기제품 (no=283)
✅ 전기제품 - 73개 종목 수집됨
🔍 크롤링 중: 석유와가스 (no=313)
✅ 석유와가스 - 20개 종목 수집됨
🔍 크롤링 중: 건축자재 (no=289)
✅ 건축자재 - 53개 종목 수집됨
🔍 크롤링 중: 비철금속 (no=322)
✅ 비철금속 - 43개 종목 수집됨
🔍 크롤링 중: 디스플레이패널 (no=327)
✅ 디스플레이패널 - 4개 종목 수집됨
🔍 크롤링 중: 사무용전자제품 (no=338)
✅ 사무용전자제품 - 1개 종목 수집됨
🔍 크롤링 중: 가정용품 (no=297)
✅ 가정용품 - 11개 종목 수집됨
🔍 크롤링 중: 전자제품 (no=307)
✅ 전자제품 - 19개 종목 수집됨
🔍 크롤링 중: 항공화물운송과물류 (no=326)
✅ 항공화물운송과물류 - 14개 종목 수집됨
🔍 크롤링 중: 인터넷과카탈로그소매 (no=308)
✅ 인터넷과카탈로그소매 - 8개 종목 수집됨
🔍 크롤링 중: 건축제품 (no=320)
✅ 건축제품 - 13개 종목 수집됨
🔍 크롤링 중: 조선 (no=291)
✅ 조선 - 28개 종목 수집됨
🔍 크롤링 중: 카드 (no=337)
✅ 카드 - 1개 종목 수집됨
🔍 크롤링 중: 생명보험 (no=330)
✅ 생명보험 - 5개 종목 수집됨
🔍 크롤링 중: 은행 (no=301)
✅ 은행 - 10개 종목 수집됨
🔍 크롤링 중: 핸드셋 (no=292)
✅ 핸드셋 - 61개 종목 수집됨
🔍 크롤링 중: 가스유틸리티 (no=312)
✅ 가스유틸리티 - 12개 종목 수집됨
🔍 크롤링 중: 

In [8]:
# 1. 산업정보 정리
df_ind = df_kospi_industry_stocks[['industry_name', 'industry_no', 'stock_name']].drop_duplicates()

# 2. 테마정보 정리 (한 종목이 여러 테마에 속할 수 있음)
df_theme = df_kospi_theme_stocks[['theme_name', 'theme_no', 'stock_name']].drop_duplicates()

# 여러 테마명을 하나로 합쳐줌
df_theme_grouped = df_theme.groupby('stock_name').agg({
    'theme_name': lambda x: ', '.join(sorted(set(x))),
    'theme_no': lambda x: ', '.join(sorted(set(x.astype(str))))
}).reset_index()

# 3. df_kospi에 산업, 테마 정보 병합
df_kospi_merged = df_kospi.copy()
df_kospi_merged = df_kospi_merged.merge(df_ind, left_on='Name', right_on='stock_name', how='left')
df_kospi_merged = df_kospi_merged.merge(df_theme_grouped, left_on = 'Name',right_on='stock_name', how='left')

### 주식 데이터 벡터화

In [9]:
# NaN은 공백 문자열로 변환 (처리 편의성)
df_kospi_merged['theme_name'] = df_kospi_merged['theme_name'].fillna('')

# theme_name을 ',' 기준으로 분리해서 list로 변환
df_kospi_merged['theme_list'] = df_kospi_merged['theme_name'].apply(lambda x: [i.strip() for i in x.split(',')] if x else [])

# 산업명과 테마 리스트를 합쳐서 하나의 list로 묶음 (멀티 레이블 형태)
df_kospi_merged['labels'] = df_kospi_merged.apply(lambda row: [row['industry_name']] + row['theme_list'], axis=1)

# MultiLabelBinarizer를 사용해서 One-hot Encoding
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
label_matrix = mlb.fit_transform(df_kospi_merged['labels'])

# 결과 데이터프레임
df_vector = pd.DataFrame(label_matrix, columns=mlb.classes_)
df_vector.insert(0, 'Name', df_kospi_merged['Name'].values)

### ETF벡터화

In [10]:
# ETF별로 구성종목 데이터프레임 나누기
etf_dict = dict(tuple(df_etf_cu.groupby('CODE')))
print(etf_dict)

# ETF 코드-이름 매핑 딕셔너리 생성
code_to_name = dict(zip(df_etf['Symbol'], df_etf['Name']))

#ETF이름으로 dict교체
etf_named_dict = {}

for code, df in etf_dict.items():
    etf_name = code_to_name[code]
    etf_named_dict[etf_name] = df


stock_vector_dict = df_vector.set_index('Name').to_dict(orient='index')
# 결과 저장용
etf_vector_list = []

# 종목 이름을 키로 한 벡터 딕셔너리

for etf_code, df_etf_comp in tqdm(etf_dict.items()):
    etf_name = code_to_name.get(etf_code, etf_code)
    
    total_weight = 0
    etf_vector = None
    included_stock_count = 0 
    
    for _, row in df_etf_comp.iterrows():
        stock_name = row['STK_NM_KOR']
        weight = row['ETF_WEIGHT']
        
        if weight == 0 or stock_name not in stock_vector_dict:
            continue
        
        stock_vec = np.array(list(stock_vector_dict[stock_name].values()))
        
        if etf_vector is None:
            etf_vector = stock_vec * weight
        else:
            etf_vector += stock_vec * weight
        
        total_weight += weight
        included_stock_count += 1 
    
    if etf_vector is not None and total_weight > 0:
        etf_vector /= total_weight  # 정규화
        etf_vector_list.append([etf_name] + etf_vector.tolist())
etf_vector = pd.DataFrame(etf_vector_list, columns=['ETF_Name'] + list(df_vector.columns[1:]))

{'0000D0':          CODE      TRD_DT  AGMT_STK_CNT  \
37559  0000D0  2025-07-29    7145330.23   
37560  0000D0  2025-07-29    5792546.70   
37561  0000D0  2025-07-29    5479994.82   
37562  0000D0  2025-07-29    4397014.23   
37563  0000D0  2025-07-29          0.00   
37564  0000D0  2025-07-29          0.00   

                                       STK_NM_KOR  ETF_WEIGHT  
37559    KEDI Nvidia U.S. 30Y Treasury Target Cov         0.0  
37560  KEDI Nvidia U.S. 30Y Treasury Target Cov_4         0.0  
37561  KEDI Nvidia U.S. 30Y Treasury Target Cov_3         0.0  
37562  KEDI Nvidia U.S. 30Y Treasury Target Cov_2         0.0  
37563                                       설정현금액         0.0  
37564                                        원화현금         0.0  , '0000H0':          CODE      TRD_DT  AGMT_STK_CNT  \
47546  0000H0  2025-07-29      33239.01   
47547  0000H0  2025-07-29      22706.88   
47548  0000H0  2025-07-29      14339.21   
47549  0000H0  2025-07-29       7955.92   
47550  0000H0

100%|██████████| 1004/1004 [00:01<00:00, 591.85it/s]


### 프로토타입을 위한 데이터 저장

In [11]:
df_vector.to_csv("data/df_vector.csv", index=False)
etf_vector.to_csv("data/etf_vector.csv", index=False)
df_customer.to_csv("data/df_customer.csv", index = False)

### 추천 작동

In [12]:
df_customer = pd.read_csv('data/df_customer.csv')
etf_vector = pd.read_csv('data/etf_vector.csv')
df_vector = pd.read_csv('data/df_vector.csv')

stock_vector_dict = df_vector.set_index('Name').to_dict(orient='index')
# =====================
# 함수 정의
def cosine_loss(weights, vectors, target_vector):
    combined = np.average(vectors, axis=0, weights=weights).reshape(1, -1)
    sim = cosine_similarity(combined, target_vector.reshape(1, -1))[0][0]
    return 1 - sim

def recommend_best_etf_combo(etf_dict, portfolio_vector, min_k=1, max_k=4):
    best_score = float('inf')
    best_combo = None
    best_weights = None

    etf_names = list(etf_dict.keys())
    for k in range(min_k, max_k + 1):
        for combo in combinations(etf_names, k):
            vectors = np.array([etf_dict[etf] for etf in combo])
            init_weights = np.ones(k) / k
            bounds = [(0, 1)] * k
            constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

            result = minimize(
                cosine_loss,
                init_weights,
                args=(vectors, portfolio_vector),
                bounds=bounds,
                constraints=constraints,
                method='SLSQP',
                options={'maxiter': 200}
            )

            if result.success and result.fun < best_score:
                best_score = result.fun
                best_combo = combo
                best_weights = result.x

    if best_combo is None:
        raise RuntimeError("최적화 실패")

    return best_combo, best_weights, 1 - best_score


def process_customer(customer_id, df_portfolios, stock_vector_dict, etf_vector, top_n=20):
    print(f"\n\n==============================")
    print(f"👤 고객 ID: {customer_id}")
    print(f"==============================")

    customer_data = df_portfolios[df_portfolios['customer_id'] == customer_id]
    total_value = customer_data['invested_value'].sum()

    print("\n📊 포트폴리오 요약:")
    for _, row in customer_data.iterrows():
        stock = row['stock_name']
        value = row['invested_value']
        pct = value / total_value * 100
        print(f"- {stock}: {value:,.0f}원 ({pct:.1f}%)")

    # 벡터화
    portfolio_vec = np.zeros(len(next(iter(stock_vector_dict.values()))), dtype=float)
    for _, row in customer_data.iterrows():
        stock = row['stock_name']
        val = row['invested_value']
        if stock not in stock_vector_dict:
            print(f"⚠️ '{stock}'는 벡터에 없음 → 무시")
            continue
        vec_data = stock_vector_dict[stock]
        if isinstance(vec_data, dict):
           vec = np.array(list(vec_data.values()), dtype=float)
        else:
           vec = np.array(vec_data, dtype=float)
    
        portfolio_vec += vec * (val / total_value)

    # ETF 벡터 딕셔너리
    etf_dict_all = {
        row['ETF_Name']: np.array(row.drop('ETF_Name').values, dtype=float)
        for _, row in etf_vector.iterrows()
    }

    # 상위 N개 ETF
    etf_sims = []
    for name, vec in etf_dict_all.items():
        sim = cosine_similarity([vec], [portfolio_vec])[0][0]
        etf_sims.append((name, sim))

    etf_sims.sort(key=lambda x: x[1], reverse=True)
    top_etf_names = [name for name, _ in etf_sims[:top_n]]
    etf_dict = {name: etf_dict_all[name] for name in top_etf_names}

    # 추천 ETF 조합
    best_combo, best_weights, best_sim = recommend_best_etf_combo(etf_dict, portfolio_vec)

    print("\n✅ 최적 추천 ETF 조합:")
    for etf in best_combo:
        print(f"- {etf}")

    print("\n📈 포트폴리오 내 ETF 비중:")
    for etf, w in zip(best_combo, best_weights):
        print(f"{etf}: {w:.4f}")

    print(f"\n🔗 유사도 점수 (Cosine Similarity): {best_sim:.4f}")


# =====================
# 실행 파트
# =====================

if __name__ == "__main__":
    # 데이터 로드
    # 고객별 처리
    customer_ids = df_customer['customer_id'].unique()
    for cid in customer_ids:
        process_customer(cid, df_customer, stock_vector_dict, etf_vector, top_n=20)



👤 고객 ID: B

📊 포트폴리오 요약:
- 카카오: 7,000,000원 (35.0%)
- 에코프로비엠: 6,000,000원 (30.0%)
- 셀트리온: 4,000,000원 (20.0%)
- HLB: 3,000,000원 (15.0%)

✅ 최적 추천 ETF 조합:
- KODEX K-뉴딜디지털플러스
- TIGER 2차전지소재Fn
- KODEX 코스닥글로벌
- TIGER BBIG

📈 포트폴리오 내 ETF 비중:
KODEX K-뉴딜디지털플러스: 0.4552
TIGER 2차전지소재Fn: 0.1148
KODEX 코스닥글로벌: 0.4287
TIGER BBIG: 0.0013

🔗 유사도 점수 (Cosine Similarity): 0.7048


👤 고객 ID: C

📊 포트폴리오 요약:
- 삼성전자: 15,000,000원 (37.5%)
- SK하이닉스: 10,000,000원 (25.0%)
- 현대차: 8,000,000원 (20.0%)
- LG화학: 7,000,000원 (17.5%)

✅ 최적 추천 ETF 조합:
- KIWOOM KRX100
- TREX 펀더멘탈 200
- PLUS 코스피50
- ACE 200

📈 포트폴리오 내 ETF 비중:
KIWOOM KRX100: 0.5682
TREX 펀더멘탈 200: 0.4168
PLUS 코스피50: 0.0149
ACE 200: 0.0000

🔗 유사도 점수 (Cosine Similarity): 0.9801


👤 고객 ID: D

📊 포트폴리오 요약:
- NAVER: 8,000,000원 (32.0%)
- 카카오: 6,000,000원 (24.0%)
- 한글과컴퓨터: 5,000,000원 (20.0%)
- 솔트룩스: 3,000,000원 (12.0%)
- 코난테크놀로지: 3,000,000원 (12.0%)

✅ 최적 추천 ETF 조합:
- TIGER 인터넷TOP10
- RISE 플랫폼테마
- RISE AI&로봇
- HANARO Fn K-메타버스MZ

📈 포트폴리오 내 ETF 비중:
TIGER 인터넷TOP10: 0.7578
RISE 플